<a href="https://colab.research.google.com/github/sznajder/Notebooks/blob/master/FwdMuonGCNHitClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph Neural Network for HIT classification

### Based on:

https://github.com/jmduarte/heptrkx-gnn-tracking/blob/master/README.md

https://github.com/jmduarte/gnn-fpga/blob/master/gnn/GCN_Toy2D.ipynb

https://github.com/jmduarte/gnn-fpga/blob/master/gnn/GCN_Seg_Toy2D.ipynb

https://github.com/jmduarte/gnn-fpga/blob/master/gnn/GCN_Toy2D_dev.ipynb

\\

There are a variety of ways to construct neural network graph operations which resemble convolutions. Here are a few possible "kernels":

1)  $\tilde{X} = \sigma(X W_0 + D X W_1 + A X W_2)$, where A is the adjacency matrix containing edge weights, D is the diagonal degree matrix, and the $W$s are learnable weight matrices. \\

2)  $\tilde{X} = \sigma(D^{-1}A X W)$, where $D^{-1}A$ is the normalized adjacency matrix. \\

3) $\tilde{X} = \sigma(\hat{D}^{-{1 \over 2}} \hat{A} \hat{D}^{-{1 \over 2}} X W)$, where $\hat{A} = A + I$ and $\hat{D}^{-{1 \over 2}} \hat{A} \hat{D}^{-{1 \over 2}}$ is the symmetrically normalized adjacency matrix.

\\

Normalizing the adjacency
A typical thing to do is $D^{-1}A$, where $D$ is the diagonal degree matrix. This amounts to normalizing all edges coming into a node so they sum to 1. Or in other words it normalizes the adjacency matrix elements by $A_{ij}= \frac{1}{\sqrt{d_i d_j}}$ , which are the the degrees of nodes $i$ and $j$ 

The approach of Kipf and Welling is $\hat{D}^{-{1 \over 2}} \hat{A} \hat{D}^{-{1 \over 2}}$, where $\hat{A} = A + I$.


In [ ]:
# System imports
from __future__ import print_function
from __future__ import division
import os
import sys
import multiprocessing as mp
from timeit import default_timer as timer

# Externals
import math
import numpy as np
import pandas as pd
from collections import namedtuple
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, find
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import cKDTree
import sklearn.metrics
from sklearn.model_selection import train_test_split

# Torch imports
import torch
from torch.autograd import Variable
import torch.nn as nn
#import torch.nn.functional as F
import torch as t

from google.colab import drive

# Mount google drive on remote Colab machine
drive.mount('/content/gdrive', force_remount=False)
sys.path.append('gdrive/My Drive/Colab Notebooks')

!ls 'gdrive/My Drive/Colab Notebooks/Data'
data_dir = 'gdrive/My Drive/Colab Notebooks/Data'

# Unzip the graphs into local /tmp
!apt install unzip
!unzip -o 'gdrive/My Drive/Colab Notebooks/Data/graphs.zip'  -d '/tmp'

# Input and Output files and events to read
events_start=0
events_end=1000


# Local imports
#from estimator import Estimator
#from acts import process_hits_files, select_barrel_hits

#%matplotlib notebook
#%matplotlib ipympl
%matplotlib inline

# Training concurrency
import os
os.environ['OMP_NUM_THREADS'] = '4'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

cuda = False

if cuda:
    np_to_torch = lambda x, volatile=False: (
        Variable(torch.from_numpy(x.astype(np.float32)), volatile=volatile).cuda())
else:
    np_to_torch = lambda x, volatile=False: (
        Variable(torch.from_numpy(x.astype(np.float32)), volatile=volatile))

torch_to_np = lambda x: x.cpu().data.numpy()

# Graph is a namedtuple of (X, Ri, Ro, y) for convenience
#feature_names = ['vh_sim_r', 'vh_sim_phi', 'vh_sim_z']
#feature_scale = np.array([1000., 360. / 6., 1000.])

feature_names = ['vh_sim_r', 'vh_sim_phi', 'vh_sim_z', 'vh_bend']
feature_scale = np.array([1000., 360. / 6., 1000.,1000.])

Graph = namedtuple('Graph', ['X', 'Ri', 'Ro', 'y_hits', 'y_segs'])

# Sparse graph uses the indices for the Ri, Ro matrices
SparseGraph = namedtuple('SparseGraph',['X', 'Ri_rows', 'Ri_cols', 'Ro_rows', 'Ro_cols', 'y_hits'])

def graph_to_sparse(graph):
    Ri_rows, Ri_cols = graph.Ri.nonzero()
    Ro_rows, Ro_cols = graph.Ro.nonzero()
    return dict(X=graph.X, y=graph.y,
                Ri_rows=Ri_rows, Ri_cols=Ri_cols,
                Ro_rows=Ro_rows, Ro_cols=Ro_cols)

def sparse_to_graph(X, Ri_rows, Ri_cols, Ro_rows, Ro_cols, y, dtype=np.uint8):
    n_nodes, n_edges = X.shape[0], Ri_rows.shape[0]
    Ri = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ro = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ri[Ri_rows, Ri_cols] = 1
    Ro[Ro_rows, Ro_cols] = 1
    return Graph(X, Ri, Ro, y_hits)



## PYTORCH module implementing a Graph Convolutional Network ( GCN )

https://github.com/jmduarte/gnn-fpga/blob/master/gnn/GCN_Seg_Toy2D.ipynb

\\

Convolutional layer definition :
$ h_i^{(l+1)} = \sigma\left(b^{(l)} + \sum_{j\in\mathcal{N}(i)}\frac{1}{c_{ij}}h_j^{(l)}W^{(l)}\right) $

, where $\mathcal{N}(i)$ is the neighboring nodes set of node $i$ and $c_{ij}$ is equal to the product of the square root of node degrees $\sqrt{|\mathcal{N}(i)|}$ and  $\sigma$  is the activation function.


In [ ]:
#def repeat_module(module, x):
#    """Applies module on last axis, repeating over others"""
#    return module(x.view(-1, x.size(-1))).view(x.size()[:-1] + (-1,))


class GraphConv(nn.Module):
    """
    A minimal graph convolution layer.
    
    This module takes an input tensor of node features X and some form
    of adjacency matrix A and applies a linear transformation of the form
        A * X * W + b
    where W and b are weights and biases, respectively.
    """
    
    def __init__(self, input_dim, output_dim):
        super(GraphConv, self).__init__()
        # We implement the patch operator as a linear module
        self.neighbor_mod = nn.Linear(input_dim, output_dim)

        # Print NN parameters and values
        print("self.neighbor_mod:")
        for name, tensor in self.neighbor_mod.named_parameters():
          print("{:6s}  -  {}".format(name, tensor.shape))
    
    def forward(self, x, a):
        ax = torch.matmul(a, x)
        h = self.neighbor_mod(ax)
        return h 


class GraphConvSelfInt(nn.Module):
    """
    A graph convolution layer with separate explicit self-interaction terms.
    
    This module takes an input tensor of node features X and adjancency
    matrix A and applies a linear transformation of the form
        X*W1 + A*X*W2 + b
    where (W1, W2) and b are learned weights and biases.
    """
    def __init__(self, input_dim, output_dim):
        super(GraphConvSelfInt, self).__init__()
        self.node_mod = nn.Linear(input_dim, output_dim)
        self.neighbor_mod = nn.Linear(input_dim, output_dim, bias=False)

        # Print NN parameters and values
        print("self.node_mod:")
        for name, tensor in self.node_mod.named_parameters():
          print("{:6s}  -  {}".format(name, tensor.shape))
        print("self.neighbor_mod:")
        for name, tensor in self.neighbor_mod.named_parameters():
          print("{:6s}  -  {}".format(name, tensor.shape))

    def forward(self, x, a):
        ax = torch.matmul(a, x)
        node_term = self.node_mod(x)
#        node_term = x
        neighbor_term = self.neighbor_mod(ax)
        h = node_term + neighbor_term
        return h



class GCNBinaryClassifier(nn.Module):

    """
    A simple graph-convolutional network for binary classification of nodes.
    
    This model applies a feature extractor to each node,
    followed by a number of graph conv layers,
    followed by a node classifier head.
    """
    
    def __init__(self, input_dim, hidden_dims, gc_type=GraphConvSelfInt):
        super(GCNBinaryClassifier, self).__init__()
        # Feature extractor layer
        self.feature_extractor = nn.Linear(input_dim, hidden_dims[0])
        # Graph convolution layers
        n_gc_layers = len(hidden_dims) - 1
        self.gc_layers = nn.ModuleList([ gc_type(hidden_dims[i], hidden_dims[i+1]) for i in range(n_gc_layers) ])        
        # Node classifier
        self.classifier = nn.Linear(hidden_dims[-1], 1)

        # Print NN parameters and values
        print("self.feature_extractor:")
        for name, tensor in self.feature_extractor.named_parameters():
          print("{:6s}  -  {}".format(name, tensor.shape))

    
    def forward(self, x, a):
        # Apply feature extraction layer
        h = self.feature_extractor(x)
        h = t.relu(h)
        # Apply graph conv layers
        for gc in self.gc_layers:
            h = gc(h, a)
            h = t.relu(h)
        # Apply node classifier
        h = self.classifier(h).squeeze(-1)
        # Applies sigmoid activation on output to normalize to [0,1]
        h = t.sigmoid(h)
        return h
        






##################################################################################

"""
# The degree matrix is computed by summing the adjacency rows
deg = np.diagflat(adj.sum(axis=0))

# Inverse degree matrix
invdeg = np.diagflat(1. / adj.sum(axis=0))

# Normalized adjacency
adj_norm = invdeg.dot(adj)
In [12]:

# Symmetric (+ identity) normalized adjacency
adj_hat = np.eye(adj.shape[0]) + adj
inv_half_deg = np.diagflat(1. / np.sqrt(adj_hat.sum(axis=0)))
adj_hat_norm = inv_half_deg.dot(adj_hat.dot(inv_half_deg))

# A*X*W
adj.dot(X).dot(W) + b

# D^-1 * A * X * W
adj_norm.dot(X).dot(W) + b

# D^(-1/2) * A * D^(-1/2) * X * W + b
adj_hat_norm.dot(X).dot(W) + b
"""

## PYTORCH module implementing the Estimator

https://github.com/jmduarte/gnn-fpga/blob/master/gnn/estimator.py

In [ ]:
"""
This module contains the Estimator class implementation which provides
code for doing the training of a PyTorch model.
"""

from __future__ import print_function

from datetime import datetime
from timeit import default_timer as timer

import numpy as np

import shutil 
import os

import torch

def logger(s):
    """Simple logger function which prints date/time"""
    print(datetime.now(), s)

class Estimator():
    """Estimator class"""

    def __init__(self, model, loss_func, opt='Adam',
                 train_losses=None, valid_losses=None,
                 cuda=False, l1=0.):

        self.model = model
        if cuda:
            self.model.cuda()
        self.loss_func = loss_func
        if opt == 'Adam':
            self.optimizer = torch.optim.Adam(self.model.parameters())
        elif opt == 'SGD':
            self.optimizer = torch.optim.SGD(self.model.parameters())

        self.train_losses = train_losses if train_losses is not None else []
        self.valid_losses = valid_losses if valid_losses is not None else []
        self.l1 = l1

        logger('Model: \n%s' % model)
        logger('Parameters: %i' %
               sum(param.numel() for param in model.parameters()))

    def l1_penalty(self, arr):
        return torch.abs(arr).sum()
        
    def training_step(self, X , A, targets):
        """Applies single optimization step on batch"""
        self.model.zero_grad()
        self.optimizer.zero_grad()
      
        outputs = self.model(X,A)

#        node_weights = [layer.weight for layer in self.model.node_network.network if hasattr(layer, 'weight')]
#        edge_weights = [layer.weight for layer in self.model.edge_network.network if hasattr(layer, 'weight')]
#        l1_regularization = self.l1 * sum([self.l1_penalty(arr) for arr in node_weights]) + self.l1 * sum([self.l1_penalty(arr) for arr in edge_weights])
#        loss = self.loss_func(outputs, targets) + l1_regularization 

#        print("X=",X.size())
#        print("A=",A.size())


        loss = self.loss_func(outputs, targets) # no regularization for the moment 
        loss.backward()
        self.optimizer.step()
        return loss

    def save_checkpoint(self, state, is_best, filename='checkpoint.pt'):
#        directory = os.path.dirname(filename)
        directory = data_dir
        try:
            os.stat(directory)
        except:
            os.mkdir(directory)
        torch.save(state, filename)
        if is_best:
            bestfilename = directory+'/model_best.pt'
            shutil.copyfile(filename, bestfilename)
            
    def load_checkpoint(self, filename='checkpoint.pt'):
        checkpoint = torch.load(filename)
        self.model.load_state_dict(checkpoint['state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.valid_losses = checkpoint['valid_losses']
        self.train_losses = checkpoint['train_losses']
    
    def fit_gen(self, train_generator, n_batches=1, n_epochs=1,
                valid_generator=None, n_valid_batches=1, verbose=0, 
                filename='checkpoint.pt'):
        """Runs batch training for a number of specified epochs."""
        epoch_start = len(self.train_losses)
        epoch_end = epoch_start + n_epochs
        if len(self.valid_losses) > 0:
            best_valid_loss = self.valid_losses[-1]
        else:
            best_valid_loss = 99999999
        for i in range(epoch_start, epoch_end):
            logger('Epoch %i' % i)
            start_time = timer()
            sum_loss = 0

            # Train the model
            self.model.train()
            
            for j in range(n_batches):
                train_x, train_a, train_y = next(train_generator)
                train_out = self.training_step(train_x, train_a, train_y)
                batch_loss = train_out.cpu().data.item() 
                sum_loss += batch_loss
                if verbose > 0:
                    logger('  Batch %i loss %f' % (j, batch_loss))
            end_time = timer()
            avg_loss = sum_loss / n_batches
            self.train_losses.append(avg_loss)
            logger('  training loss %.3g time %gs' %
                   (avg_loss, (end_time - start_time)))

            with torch.no_grad():
                # Evaluate the model on the validation set
                if (valid_generator is not None) and (n_valid_batches > 0):
                    self.model.eval()
                    valid_loss = 0
                    for j in range(n_valid_batches):
                        valid_x, valid_a, valid_y = next(valid_generator)
                        valid_loss += (self.loss_func(self.model(valid_x, valid_a ), valid_y).cpu().data.item())
                    valid_loss = valid_loss / n_valid_batches
                    self.valid_losses.append(valid_loss)
                    logger('  validate loss %.3g' % valid_loss)
                
                    #Save model checkpoint - modified
                    logger(' save checkpoint') 
                    is_best = valid_loss < best_valid_loss
                    best_valid_loss = min(valid_loss, best_valid_loss)
                    self.save_checkpoint({
                        'epoch': i + 1,
                        'state_dict': self.model.state_dict(),
                        'best_valid_loss': best_valid_loss,
                        'valid_losses': self.valid_losses,
                        'train_losses': self.train_losses,
                        'optimizer' : self.optimizer.state_dict(),
                    }, is_best, filename=filename)

    def predict(self, generator, n_batches, concat=True):
        with torch.no_grad():  
            self.model.eval()
            outputs = []
            for j in range(n_batches):
                test_x, test_a,  test_y = next(generator)
                outputs.append(self.model(test_x, test_a))
            if concat:
                outputs = torch.cat(outputs)
            return outputs

 

 ##############################################################################

# Estimator for GCN
'''
def predict_prob(model, inputs):
     return t.sigmoid(model(inputs))

def training_step(model, inputs, targets, loss_func, optimizer):
    model.train()
    model.zero_grad()
    outputs = model(*inputs)
    loss = loss_func(outputs, targets)
    loss.backward()
    optimizer.step()
    return loss

def accuracy(probs, target, threshold=0.5):
    return ((probs.data.numpy() > threshold) == (target.data.numpy() > 0.5)).mean()
'''

## Batch Generator

In [ ]:
#def batch_generator(X, Ri, Ro, y, n_samples=1, batch_size=1, train=True):
def batch_generator(X, A, y, n_samples=1, batch_size=1, train=True):
    volatile = not train
    batch_idxs = np.arange(0, n_samples, batch_size)
    # Loop over epochs
    while True:
        # Loop over batches
        for j in batch_idxs:
# Comented lines bellow because volatile is deprecated
#            batch_X  = np_to_torch(  X[j:j+batch_size], volatile=volatile )
#            batch_Ri = np_to_torch( Ri[j:j+batch_size], volatile=volatile )
#            batch_Ro = np_to_torch( Ro[j:j+batch_size], volatile=volatile )
#            batch_y  = np_to_torch(  y[j:j+batch_size], volatile=volatile )
 
          with torch.set_grad_enabled(train):
            batch_X  = np_to_torch(  X[j:j+batch_size] )
            batch_A  = np_to_torch( A[j:j+batch_size] )
            batch_y  = np_to_torch( y[j:j+batch_size] )
#            batch_Ri = np_to_torch( Ri[j:j+batch_size] )
#            batch_Ro = np_to_torch( Ro[j:j+batch_size] )
#            batch_inputs = [batch_X, batch_Ri, batch_Ro]
            yield batch_X, batch_A , batch_y

## Network  Model and Training Configuration

In [ ]:
# Model achitecture 
n_hidden = [10,10,10,10,10] # number of features in each node of a graph convolutional hidden layer

# Training config
batch_size = 50
n_epochs = 100
valid_frac = 0.2
test_frac = 0.2


## Load and prepare the graphs DATASET

In [ ]:
# Load GRAPHS from inputfile
#sparse = np.load(infile, allow_pickle=True)
#gfile = np.load(infile)
#garray = gfile.f.arr_0 # convert file to 2D np.ndarrays
import glob
#filenames =  glob.glob(data_dir+'/graphs/*.npz')
filenames =  glob.glob('/tmp/graphs/*.npz')
#print("Filenames=",filenames)

# Load the graphs into a list 
graphs = []
for f in filenames:
  gfile = np.load(f)
#  graph = SparseGraph(**dict(gfile.items()))
  graph = Graph(**dict(gfile.items()))
  graphs.append(graph)

# Get the matrix sizes in this batch
n_graphs = len(graphs)
n_features = graphs[0].X.shape[1] 
n_nodes    = np.array([g.X.shape[0] for g in graphs])
n_edges    = np.array([g.y_segs.shape[0] for g in graphs])
n_hits    = np.array([g.y_hits.shape[0] for g in graphs])
max_nodes = n_nodes.max()
max_edges = n_edges.max()

'''
print("n_graphs",n_graphs)
print("n_features",n_features)
print("n_nodes",n_nodes)
print("n_edges",n_edges)
print("n_hits",n_hits)
print("max_nodes",max_nodes)
print("max_edges",max_edges)
'''

# Define GRAPH tensors for the full dataset
n_samples = n_graphs
X  = np.zeros((n_samples, max_nodes, n_features), dtype=np.float32) # node features 
Ri = np.zeros((n_samples, max_nodes, max_edges) , dtype=np.float32)  # adjacency matrix
Ro = np.zeros((n_samples, max_nodes, max_edges) , dtype=np.float32)  #
y_hits  = np.zeros((n_samples, max_nodes), dtype=np.float32)             # target label
y_segs  = np.zeros((n_samples, max_edges), dtype=np.float32)             # target label
A  = np.zeros((n_samples, max_nodes , max_nodes), dtype=np.float32 )

# Loop over graphs and fill the NN input tensors ( with event # indexed  by i )
for i, g in enumerate(graphs):
  X[i,  :n_nodes[i]] = g.X 
  Ri[i, :n_nodes[i], :n_edges[i]] = g.Ri
  Ro[i, :n_nodes[i], :n_edges[i]] = g.Ro
  y_hits[ i, :n_nodes[i] ] = g.y_hits                 
  y_segs[ i, :n_edges[i] ] = g.y_segs 

  # Fill the ADJACENCY matrix from edge indexes in COO representation
  for iedge in range(max_edges):
    ri = np.flatnonzero( Ri[i,:,iedge] )
    ro = np.flatnonzero( Ro[i,:,iedge] )
#    print( "Shit ------>>>> len(ri),len(ro)= ",len(ri),len(ro) )
#    print( "iedge= ",iedge)
#    print( "ri= ",ri)
#    print( "ro= ",ro)
#    print(" ")
    if (len(ri)==1 and len(ro)==1):  A[i,ri[0],ro[0]]=1.


## Partition dataset into TRAIN , TEST and VALIDATION

In [ ]:
%%time
# We round by batch_size to avoid partial batches
n_test  = int(n_samples * test_frac)     // batch_size * batch_size
n_valid = int(n_samples * valid_frac)    // batch_size * batch_size
n_train = (n_samples - n_valid - n_test) // batch_size * batch_size
n_train_batches = n_train // batch_size
n_valid_batches = n_valid // batch_size
n_test_batches  = n_test  // batch_size

# Partition the dataset into TEST, VALIDATION and TRAIN 
#(train_X, test_X,  train_A, test_A,  train_y, test_y)  = train_test_split(X, A, y, test_size=n_test)
#(train_X, valid_X, train_A, valid_A, train_y, valid_y) = train_test_split(X, A, y, test_size=n_valid)
(train_X, test_X,  train_A, test_A,  train_y, test_y)  = train_test_split(X, A, y_hits, test_size=n_test)
(train_X, valid_X, train_A, valid_A, train_y, valid_y) = train_test_split(X, A, y_hits, test_size=n_valid)

# Prepare the batch_generator tortch samples
train_batcher = batch_generator(train_X, train_A, train_y, train=True , n_samples=n_train, batch_size=batch_size)
valid_batcher = batch_generator(valid_X, valid_A, valid_y, train=False, n_samples=n_valid, batch_size=batch_size)
test_batcher  = batch_generator(test_X ,  test_A, test_y , train=False, n_samples=n_test , batch_size=batch_size)

'''
print('Graphs shapes:', X.shape , Ri.shape , Ro.shape , y_hits.shape)
print('Graphs node features:', feature_names)
print("n_train, n_valid, n_test  = " , n_train, " , " , n_valid, " , " , n_test )
print('Train shapes:', train_X.shape , train_A.shape , train_y.shape)
print('Valid shapes:', valid_X.shape , valid_A.shape , valid_y.shape)
print('Test shapes: ', test_X.shape  , test_A.shape  , test_y.shape)
'''


## Construct the GNN model

In [ ]:
# Construct the model
print("n_features=",n_features)
print("n_hidden=",n_hidden)
model = GCNBinaryClassifier(n_features, n_hidden,gc_type=GraphConvSelfInt)
loss_func = nn.BCELoss()
estim = Estimator(model, loss_func=loss_func, cuda=cuda)

## Train the GNN

In [ ]:
estim.fit_gen(train_batcher, n_batches=n_train_batches, n_epochs=n_epochs,valid_generator=valid_batcher, n_valid_batches=n_valid_batches)

## Evaluate NN Training and Performance


In [ ]:

from sklearn.metrics import accuracy_score , precision_score , recall_score , precision_recall_curve , roc_curve, auc
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('default')
plt.figure(figsize=(15,10))
plt.legend(fontsize=10)
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
LINE_WIDTH = 2

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# plot loss vs epoch
ax = plt.subplot(3, 2, 1)
ax.plot(estim.train_losses, label='training set',lw=LINE_WIDTH)
ax.plot(estim.valid_losses, label='validation set',lw=LINE_WIDTH)
#ax.set_ylim([0, 1])
ax.legend(loc="upper right")
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')

'''
# plot accuracy vs epoch
ax = plt.subplot(3, 2, 2)
ax.plot(estim.train_accuracy, label=='training set',lw=LINE_WIDTH)
ax.plot(estim.valid_accuracy, label='validation set',lw=LINE_WIDTH)
#ax.set_ylim([0, 1])
ax.legend(loc="upper right")
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
'''


# Evaluate on TEST data
test_outputs = estim.predict(test_batcher, n_test_batches, concat=False)
test_pred = torch_to_np(estim.predict(test_batcher, n_test_batches))

flat_y = test_y.flatten()
flat_pred = test_pred.flatten()

# Compute the ROC and Precisio X Recall curve
fpr, tpr, _ = sklearn.metrics.roc_curve(flat_y, flat_pred)
roc_auc = sklearn.metrics.auc(fpr, tpr)
p, r, t = sklearn.metrics.precision_recall_curve(flat_y, flat_pred)




# NN discriminatnt CUT 
cut=0.5

# Transform predictions into a array of entries 0,1 depending if prdiction is above cut
y_pred = flat_y.copy()
y_pred[y_pred >= cut]=1
y_pred[y_pred < cut]=0
y_true = flat_y.copy()
w_test = np.ones(len(y_pred))

print("y_true.shape",y_true.shape)
print("y_pred.shape",y_pred.shape)
print("w_test.shape",w_test.shape)

print("y_true",y_true)
print("y_pred",y_pred)
print("w_test",w_test)

accuracy  = accuracy_score(y_true, y_pred, sample_weight=w_test)
precision = precision_score(y_true, y_pred, sample_weight=w_test)
recall    = recall_score(y_true, y_pred, sample_weight=w_test)
print('DNN output cut:      %.4f' % cut)
print('Accuracy:            %.4f' % accuracy)
print('Precision/Purity:  %.4f' % precision)
print('Sensitivity/Recall/TPR/Signal Efficiency: %.4f' % recall)
#print('Specificity/Selectivity/TNR/Background Efficiency: %.4f' % recall)

 

# Plot ROC
roc_auc = auc(fpr, tpr)
ax = plt.subplot(3, 2, 2)
ax.plot(fpr, tpr, lw=LINE_WIDTH, color='cyan', label='auc = %.3f' % (roc_auc))
ax.plot([0, 1], [0, 1], linestyle='--', color='k', label='random chance')
ax.set_xlim([0, 1.0])
ax.set_ylim([0, 1.0])
ax.set_xlabel('False Positive Rate(FPR)')
ax.set_ylabel('True Positive Rate(TPR)')
#ax.set_title('Receiver Operating Curve(ROC)')
ax.legend(loc="lower right")

# Get model prediction for signal and background 
mask = y_true.astype(int)
y_sig = flat_pred[mask==1]
y_bkg = flat_pred[mask==0]

ax = plt.subplot(3, 2, 3)
X = np.linspace(0.0, 1.0, 100)
hist_sig = ax.hist(y_sig, bins=X, label='sig',histtype='step',lw=LINE_WIDTH)
#hist_bkg = ax.hist(Y_bkg, bins=X, label='bkg',histtype='step',lw=LINE_WIDTH)
#ax.hist(Y_train_val, bins=X, label='bkg',histtype='step')
ax.set_xlabel('DNN Output for SGN')
ax.legend(prop={'size': 10})

ax = plt.subplot(3, 2, 4)
X = np.linspace(0.0, 1.0, 100)
#hist_sig = ax.hist(Y_sig, bins=X, label='sig',histtype='step',lw=LINE_WIDTH)
hist_bkg = ax.hist(y_bkg, bins=X, label='bkg',histtype='step',lw=LINE_WIDTH)
#ax.hist(Y_train_val, bins=X, label='bkg',histtype='step')
ax.set_xlabel('DNN Output for BKG')
ax.legend(prop={'size': 10})



# Plot Eff x Purity

ax = plt.subplot(3, 2, 5)
ax.plot(t, p[:-1], label='purity', lw=LINE_WIDTH)
ax.plot(t, r[:-1], label='efficiency', lw=LINE_WIDTH)
ax.set_xlabel('Cut on model score')
ax.tick_params(width=2, grid_alpha=0.5)
ax.legend()

ax = plt.subplot(3, 2, 6)
ax.plot(t,p[:-1]*r[:-1], lw=LINE_WIDTH)
ax.set_xlabel('Cut on model score')
ax.set_ylabel('Purity*Efficiency')
ax.tick_params(width=2, grid_alpha=0.5)

# Show plots
plt.show()




## Visualize some samples

In [ ]:
def drawGraph(X, A, y, pred): 
    # Select the i/o node features for each segment    
    # Prepare the figure
 
    fig, (ax0,ax1) = plt.subplots(1, 2, figsize=(12,8))
#    cmap = plt.get_cmap('bwr_r')

    # Rescale for plotting purposes
    X=X*feature_scale
    # HITS features 
    R=X[:,0]
    Phi=(np.pi/180.)*X[:,1]
    Z=X[:,2] # Don't save isMuon X[:,3] component in graph feature
    XX=R*np.cos(Phi)
    YY=R*np.sin(Phi)

    # Transform 0 and 1 labels into color list using the dictionary 
    colordic={1:'blue' , 0:'red'}
#    cl=[colordic.get(n) for n in X[:,3]] # use the hit target value for color

    # Plot the hits
    ax0.scatter(XX,YY, s=100., color='black' )
    ax1.scatter(Z,R, s=100., color='black' )
    
    # Get the hits indices from the SGMENTS arrays
    idx_i = find(np.rot90(Ri))[1]
    idx_o = find(np.rot90(Ro))[1]
  
    # Reverse to get correct hits order as in dh_hits 
    idx_i=idx_i[::-1]
    idx_o=idx_o[::-1]

    # Get SEGMENTS begin and end hits coordinates
    R_i=X[idx_i,0]
    Phi_i=(np.pi/180.)*X[idx_i,1]
    Z_i=X[idx_i,2]
    X_i=R_i*np.cos(Phi_i)
    Y_i=R_i*np.sin(Phi_i)
  
    R_o=X[idx_o,0]
    Phi_o=(np.pi/180.)*X[idx_o,1]
    Z_o=X[idx_o,2]
    X_o=R_o*np.cos(Phi_o)
    Y_o=R_o*np.sin(Phi_o)
  
    # Create a list of SEGMENTS colors 
#    cl=[colordic.get(n) for n in y]
    cl=[colordic.get(np.round((n-thresh+0.5),0)) for n in pred]

    # Plot segments

    for j in range(len(X_i)):
      ax0.plot([X_i[j], X_o[j]], [Y_i[j], Y_o[j]], '-', color=cl[j])
      ax1.plot([Z_i[j], Z_o[j]], [R_i[j], R_o[j]], '-', color=cl[j])


    # Show plots
    ax0.set_xlabel('X')
    ax0.set_ylabel('Y')
    ax1.set_xlabel('Z')
    ax1.set_ylabel('R')
    plt.show()


# Loop to draw one graph per event

for i in range(n_test):
  
    X = test_X[i,:,:] 
    A = test_A[i,:]
    Ro = test_Ro[i,:]

    y = flat_y[i] 
    pred = flat_pred[i] 

    print('Y =',y)
    print('PRED =',pred)
    print('--------------------------------------')

    print('accuracy %.3f, precision %.3f, recall %.3f' % (
        sklearn.metrics.accuracy_score(y, pred>thresh),
        sklearn.metrics.precision_score(y, pred>thresh),
        sklearn.metrics.recall_score(y, pred>thresh)))

    drawGraph(X, A, y , pred);